In [ ]:
import mod_sql
import pandas as pd

: 

In [ ]:
# mods_sql을 통해서 Database 클래스를 생성
db = mod_sql.Database("localhost", "root", "ubion7", 3306)

: 

In [ ]:
sql = """
    select 
    `Country` as '국가', 
    `Units Sold` as '판매개수', 
    `Unit Price` as '가격', 
    `Unit Cost` as '원가', 
    (`Units Sold` * `Unit Price`) as '총수익', 
    (`Units Sold` * `Unit Cost`) as '총비용', 
    (`Units Sold` * `Unit Price` - `Units Sold` * `Unit Cost`) as '총이윤'
    from 
    `Sales Records`
"""
db.sql_query(sql)

: 

In [ ]:
df = pd.read_csv('../csv/Sales Records.csv')

: 

In [ ]:
df.drop(['Region', 'Item Type', 'Sales Channel', 'Order Priority', 'Order Date', 'Order ID', 'Ship Date'], axis = 1, inplace = True)
#df = df[['Country', 'Units Sold', 'Unit Price', 'Unit Cost']]

: 

In [ ]:
df.columns = ['국가', '판매개수', '가격', '원가', '총수익', '총비용', '총이윤']
df

: 

In [ ]:
df.drop(['총수익', '총비용', '총이윤'], axis = 1, inplace=True)

: 

In [ ]:
df['총수익'] = df['판매개수'] * df['가격']
df['총비용'] = df['판매개수'] * df['원가']
df['총이윤'] = (df['판매개수'] * df['가격']) - (df['판매개수'] * df['원가'])
#df['총이윤'] = df['총수익'] - df['총비용']
df

: 

### sql에서 반올림

In [ ]:
sql = """
    select
    `Total Cost`,
    round(`Total Cost`, 1)
    from
    `Sales Records`
"""

db.sql_query(sql)

: 

## sql 그룹화
1. Country, Item Type을 기준으로 그룹화
2. unit sold의 평균 합계 생성
3. Country를 기준으로 오름차순 정렬

In [ ]:
sql = """
    select
    `Country`,
    `Item Type`,
    AVG(`Units Sold`) as AVG,
    SUM(`Units Sold`) as SUM
    from
    `Sales Records`
    group by
    `Country`,
    `Item Type`
    order by
    `Country`
    ASC
"""

db.sql_query(sql)

: 

In [ ]:
df = pd.read_csv('../csv/Sales Records.csv')

: 

In [ ]:
df.groupby(['Country', 'Item Type']).agg(['mean', 'sum'])['Units Sold'].reset_index()

: 

### dept table에서 부서의 이름(dname)이 sales인 소속 사원의 정보 출력
1. dept 테이블에서 부서이름이 sales인 deptno를 출력
2. emp 테이블에서 1번 과정에서 나온 deptno를 기준으로 사원 출력

In [ ]:
sql = """
    select
    `deptno`
    from
    `dept`
    where
    `dname` = 'sales'
"""

db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    `emp`
    where
    `deptno` = 30
"""
db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    `emp`
    where
    `deptno` = (
    select
    `deptno`
    from
    `dept`
    where
    `dname` = 'sales'
    )
"""
db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    `emp`
    where
    `deptno` = (
        select
        `deptno`
        from
        `dept`
        where
        `dname` = 'research'
    )
"""
db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    `deptno`
    from
    `dept`
    where
    `dname` in ('sales', 'research')
"""

db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    `emp`
    where
    `deptno` in (
    select
    `deptno`
    from
    `dept`
    where
    `dname`
    in ('sales', 'research'))
"""

db.sql_query(sql)

: 

In [ ]:
emp = pd.read_csv('../csv/emp.csv')
dept = pd.read_csv('../csv/dept.csv')

: 

In [ ]:
flag = dept.loc[(dept['DNAME'].isin(['RESEARCH', 'SALES'])), 'DEPTNO'].tolist()

: 

In [ ]:
emp.loc[emp['DEPTNO'].isin(flag)].reset_index(drop=True)

: 

1. emp 데이터프레임에서 JOB 컬럼의 데이터를 확인하여 새로운 파생변수 생성
2. job_code SALESMAN이면 1, MANAGER면 2, 나머지는 3으로 하여 새로운 파생변수에 대입

In [ ]:
emp['JOB']

: 

In [ ]:
emp['job_code'] = emp['JOB'].apply(lambda x : 1 if (x == 'SALESMAN') else (2 if (x == 'MANAGER') else 3))
emp

: 

In [ ]:
for i in emp.index:
    if emp.loc[i, 'JOB'] == 'SALESMAN':
        emp.loc[i, 'job_code'] = 1
    elif emp.loc[i, 'JOB'] == 'MANAGER':
        emp.loc[i, 'job_code'] = 2
    else:
        emp.loc[i, 'job_code'] = 3
emp

: 

In [ ]:
def change (x):
    if x == 'SALESMAN':
        result = 1
    elif x == 'MANAGER':
        result = 2
    else:
        result = 3
    return result

emp['job_code'] = emp['JOB'].apply(change)
emp

: 

In [ ]:
sql = """
    select
    `EMPNO`,
    `ENAME`,
    `JOB`,
    CASE
        WHEN `JOB` = 'SALESMAN'
        THEN 1
        WHEN `JOB` = 'MANAGER'
        THEN 2
        ELSE 3
    END as 'job_code'
    from
    emp
"""

db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    `EMPNO`,
    `ENAME`,
    `JOB`,
    if(
        `JOB` = 'SALESMAN',
        1,
        if(`JOB` - 'MANAGER',
            2,
            3
        )
    )as `job_code`
    from
    emp 
"""
db.sql_query(sql)

: 

1. emp 데이터에서 파생변수 H/L 생성
2. SAL이 2000 이상이면 High, 아니면 Low 데이터를 삽입

In [ ]:
for i in emp.index:
    if emp.loc[i, 'SAL'] >= 2000 :
        emp.loc[i, 'H/L'] = 'High'
    else:
        emp.loc[i, 'H/L'] = 'Low'

: 

In [ ]:
emp['H/L'] = emp['SAL'].apply(lambda x : 'High' if (x >= 2000) else 'Low')

: 

In [ ]:
def change (x):
    if x >= 2000:
        result = 'High'
    else:
        result = 'Low'
    return result

emp['H/L'] = emp['SAL'].apply(change)
emp

: 

In [ ]:
sql = """
    select
    `EMPNO`,
    `ENAME`,
    `SAL`,
    CASE
        WHEN `SAL` >= 2000
        THEN 'High'
        ELSE 'Low'
    END as 'H/L'
    from
    emp
"""

db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    `EMPNO`,
    `ENAME`,
    `SAL`,
    if(
        `SAL` >= 2000,
        'High',
        'Low'
    )as 'H/L'
    from
    emp 
"""
db.sql_query(sql)

: 

### join
1. emp 와 dept 테이블 결합

In [ ]:
sql = """
    select
    *
    from
    emp
    left join
    dept
    on
    emp.DEPTNO = dept.DEPTNO
"""

db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    emp.EMPNO,
    emp.ENAME,
    emp.DEPTNO,
    dept.DNAME,
    dept.LOC
    from
    emp
    left join
    dept
    on
    emp.DEPTNO = dept.DEPTNO
"""

db.sql_query(sql)

: 

In [ ]:
pd.merge(emp, dept, on='DEPTNO', how='left')


: 

In [ ]:
pd.merge(emp, dept, on='DEPTNO', how='right')

: 

### 문제
- 부서의 위치가 'NEW YORK', 'DALLAS'가 아닌 소속 사원의 정보를 출력

In [ ]:
sql = """
    select
    *
    from
    dept
    where
    `LOC` not in ('NEW YORK', 'DALLAS')
"""

db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    dept
    where
    LOC != 'DALLAS' AND LOC != 'NEW YORK'
"""
db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    emp
    where
    DEPTNO = 30
    OR
    DEPTNO = 40
"""

db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    emp
    where
    DEPTNO in (30, 40)
"""

db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    emp
    where
    DEPTNO in (
        select
        DEPTNO
        from
        dept
        where
        LOC not in ('NEW YORK', 'DALLAS')
)"""

db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    emp
    left join
    dept on
    emp.DEPTNO = dept.DEPTNO
    where
    LOC not in('NEW YORK', 'DALLAS'
)"""

db.sql_query(sql)

: 

### 테이블의 결합
1. 유니언 결합(단순하게 행을 결합)
2. 조인 결합

In [ ]:
sql = """
    select
    *
    from
    tran_1
"""
db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    tran_2
"""
db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    tran_1
    union
    select
    *
    from
    tran_2
"""
db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    tran_d_1
    union
    select
    *
    from
    tran_d_2
"""
db.sql_query(sql)

: 

In [ ]:
sql = """
    select
    *
    from
    tran_1
    left join
    tran_d_1
    on tran_1.transaction_id = tran_d_1.transaction_id
"""
db.sql_query(sql)

: 

In [ ]:
sql = """
    select * from
    (select * from tran_1
    union
    select * from tran_2) as tran
    left join
    (select * from tran_d_1
    union
    select * from tran_d_2) as tran_d
    on tran.transaction_id = tran_d.transaction_id
"""

db.sql_query(sql)

: 

In [ ]:
tran1 = pd.read_csv('../csv/tran_1.csv')
tran2 = pd.read_csv('../csv/tran_2.csv')
tran_d1 = pd.read_csv('../csv/tran_d_1.csv')
tran_d2 = pd.read_csv('../csv/tran_d_2.csv')

: 

In [ ]:
df = pd.concat([tran1, tran2], axis=0)

: 

In [ ]:
df2 = pd.concat([tran_d1, tran_d2], axis=0)

: 

In [ ]:
df3 = pd.merge(df, df2, left_on = 'transaction_id', right_on = 'transaction_id', how = 'left')

: 

In [ ]:
df3

: 

### 문자열이 포함되어 있는 경우 출력

In [ ]:
sql = """
    select
    *
    from
    emp
    where
    ENAME like '%%S%%'
"""

db.sql_query(sql)

: 

1. pandas 이용하여 emp.csv에서 ENAME이 S로 시작하는 데이터를 출력
2. s로 끝나는 데이터를 출력
3. s가 포함되어 있는 데이터를 출력

In [ ]:
# case1
emp[emp['ENAME'].str.startswith('S')]

: 

In [ ]:
# case2
data = pd.DataFrame()
for i in emp.index:
    if emp.loc[i, 'ENAME'].startswith('S'):
        data = pd.concat([data, emp.loc[[i]]])

data

: 

In [ ]:
#case3
_list = []
for i in emp.index:
    if emp.loc[i, 'ENAME'].startswith('S'):
        _list.append(i)

emp.loc[_list]

: 

In [ ]:
emp[emp['ENAME'].str.endswith('S')]

: 

In [ ]:
emp.loc[emp['ENAME'].apply(lambda x : x.find('S') == len(x)-1)]

: 

In [ ]:
emp[emp['ENAME'].str.contains('S')]

: 

In [ ]:
emp.loc[emp['ENAME'].apply(lambda x : x.find('S') != -1)]

: 

In [ ]:
emp.loc[emp['ENAME'].apply(lambda x: ('S' in x))]

: 

: 